In [1]:
# 필요한 기본 패키지 준비

# 데이터 처리 필요 패키지
import numpy as np
import pandas as pd
import datetime as dt

# 시각화 필요 패키지
%matplotlib inline
from plotnine import *
import folium
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import rc, font_manager
import seaborn as sns


# Machine Learning 분석 환경 준비

# 전처리, 스케일링
from sklearn.preprocessing import StandardScaler

# 회귀분석
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from scipy import stats

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error 
from xgboost import XGBRegressor
import statsmodels.api as sm
#데이터셋 분리
from sklearn.model_selection import train_test_split

# 다중공선성(multicollinearity) 처리를 위한VIF 확인 패키지
from statsmodels.stats.outliers_influence import variance_inflation_factor


# 한글 처리
font_name = font_manager.FontProperties(fname='C:/Windows/Fonts/NanumGothicCoding.ttf').get_name()
rc('font',family=font_name)

# - 마이너스 사인 처리
matplotlib.rcParams['axes.unicode_minus'] = False

# jupyter notebook에서 warning 무시하기
import warnings
warnings.filterwarnings("ignore")
warnings.filterwarnings('ignore', category=DeprecationWarning)

C:\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [38]:
#########################################################################
# MAD 기반 예제코드
def mad_based_outlier(points, thresh=3.5):
    if len(points.shape) == 1:
        points = points[:,None]
    median = np.median(points, axis=0)
    diff = np.sum((points - median)**2, axis=-1)
    diff = np.sqrt(diff)
    med_abs_deviation = np.median(diff)
    modified_z_score = 0.6745 * diff / med_abs_deviation
    return modified_z_score > thresh 

# 출처: https://pythonanalysis.tistory.com/7 [Python 데이터 분석]
#########################################################################

# 소셜 데이터 처리를 위한 함수
# 1. 모든 소셜 데이터 column들의 첫번째는 : 날짜다.
# 2. 각 소셜데이터는 social_키워드.블로그/트위터/뉴스/총합 으로 되어 있다.
def changeColNames(d,to_replace, replaced) : 
    # 컬럼이름 리스트를 만들어 반환
    # 통합하기 쉽게, 모든 데이터들의 날짜컬럼 이름을 date로 통일
    new_col_names = ['date']
    new_col_names.extend(list(d.columns)[1:])
    d.columns = new_col_names
    return pd.Series(d.columns).apply(lambda x : x.replace(to_replace,replaced))


# ['temp','humid','wind','rain','snow','cloud','sun_time'
#                  ,'pm.total', 'health.total','br.total', 'hobby.total','date.total']
# modeling 함수로 만들어 처리하기
def linReg(df, item, cols_using, score=False):
    cols = cols_using
    X = df.loc[df['category']==item, cols]
    y = df.loc[df['category']==item,'qty']

    X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3, random_state=0)

    model = LinearRegression().fit(X_train, y_train)
  
    print('LinearRegression을 이용한 %s의 회귀분석 결과 :'%item)
    print('훈련세트점수 : {:.2f}'.format(model.score(X_train, y_train)))
    print('검증세트점수 : {:.2f}'.format(model.score(X_test, y_test)))
    if score ==True:
        return model.score(X_train, y_train), model.score(X_test, y_test)
    
def ridgeReg(df, item, cols_using):
    cols = cols_using
    X = df.loc[df['category']==item,cols]
    y = df.loc[df['category']==item,'qty']

    X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3, random_state=0)

    ridge = Ridge(alpha=0.1, normalize=True, random_state=0, tol=0.001).fit(X_train, y_train)
    
    print('RidgeRegression을 이용한 %s의 회귀분석 결과 :'%item)
    print('훈련세트점수 : {:.2f}'.format(ridge.score(X_train, y_train)))
    print('검증세트점수 : {:.2f}'.format(ridge.score(X_test, y_test)))
    if score ==True:
        return model.score(X_train, y_train), model.score(X_test, y_test)

def lassoReg(df, item, cols_using):
    cols = cols_using
    X = df.loc[df['category']==item,cols]
    y = df.loc[df['category']==item,'qty']

    X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3, random_state=0)

    lasso = Lasso(alpha=0.1, max_iter=1000).fit(X=X_train, y=y_train)
  
    print('LassoRegression을 이용한 %s의 회귀분석 결과 :'%item)
    print('훈련세트점수 : {:.2f}'.format(lasso.score(X_train, y_train)) )
    print('검증세트점수 : {:.2f}'.format(lasso.score(X_test, y_test)) )

    #사용한 특성수
    print('사용한 특성수 : {}'.format(np.sum(lasso.coef_ != 0)) )
    if score ==True:
        return model.score(X_train, y_train), model.score(X_test, y_test)
####################################################    
def rfReg(df, item, cols_using):
    cols = cols_using
    X = df.loc[df['category']==item,cols]
    y = df.loc[df['category']==item,'qty']
    
    X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3, random_state=0)
    
    model = RandomForestRegressor()
    model.fit(X_train,y_train)
    # Get the mean absolute error on the validation data
    RFpredicted = model.predict(X_test)
    MAE = mean_absolute_error(y_test , RFpredicted)
    print('Random forest validation MAE = ', MAE)
    print('훈련세트점수 : {:.2f}'.format(model.score(X_train, y_train)))
    print('검증세트점수 : {:.2f}'.format(model.score(X_test, y_test)))
    if score ==True:
        return model.score(X_train, y_train), model.score(X_test, y_test)
    
def xgbReg(df, item, cols_using):
    cols = cols_using
    X = df.loc[df['category']==item, cols]
    y = df.loc[df['category']==item,'qty']

    X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3, random_state=0)
    XGBModel = XGBRegressor(objective='reg:squarederror')
    XGBModel.fit(X_train,y_train , verbose=False)
    # Get the mean absolute error on the validation data :
    XGBpredictions = XGBModel.predict(X_test)
    MAE = mean_absolute_error(y_test , XGBpredictions)
    print('XGBoostRegression을 이용한 %s의 회귀분석 결과 :'%item)
    print('XGBoost validation MAE = ',MAE)
    print('훈련세트점수 : {:.2f}'.format(XGBModel.score(X_train, y_train)))
    print('검증세트점수 : {:.2f}'.format(XGBModel.score(X_test, y_test)))
    if score ==True:
        return model.score(X_train, y_train), model.score(X_test, y_test)
    
    ###################################
def addDayWeek(df):
    df_work = df.copy()
    df_work['day'] = pd.Series(range(1,df_work.shape[0]+1))
    df_work['week'] = df_work['day'].apply(lambda x : x//7)
    return df_work
    
def mergeForAnalysis(df1, df2, df3=None, item='아이스크림'):
    merged_df = pd.merge(df1.loc[df1.category==item], df2, on='date',how='left')
    if df3 is not None:
        merged_df = pd.merge(merged_df, df3, on='date', how='left')
    return merged_df
#######################
def lowVIF(df, n=7, cols_using =['temp', 'cloud', 'wind','humid', 'hpa', 'sun_time', 'lgt_time', 
       'SO2', 'CO', 'O3', 'NO2', 'PM10', 'PM25'] ):
    col_to_use = cols_using
    vif = pd.DataFrame()
    vif["VIF_Factor"] = [variance_inflation_factor(
        df[col_to_use].values, i) for i in range(df[col_to_use].shape[1])]
    vif["features"] = col_to_use
    vif.sort_values("VIF_Factor")
    lowest_vif = vif.sort_values("VIF_Factor")[:n].reset_index()
    lowest_vif.drop(columns='index', inplace=True)
    return lowest_vif

#########################################################################
# ols모델용 formula 생성
def formulaGen(target, ind_features):
    '''
    formulaGen(목표컬럼명,[변수컬럼명1, 변수컬럼명2,...])
    '''
    custom_formula = target + " ~ "
    for f in range(len(ind_features)):
        custom_formula += ind_features[f]
        if f!=(len(ind_features)-1):
            custom_formula += " + "
    return custom_formula


#########################################################################

In [3]:
# 데이터 불러오기 (전처리 된 GS, 랄라블라, 날씨)
gs = pd.read_csv('d:/project/contest/data/processed/p_gs.csv', parse_dates=['date'])
lv = pd.read_csv('d:/project/contest/data/processed/p_lavla.csv', parse_dates=['date'])
w = pd.read_csv('d:/project/contest/data/processed/p_wUVair_seoul.csv', parse_dates=['date'], index_col=0)
sns_all = pd.read_csv('d:/project/contest/data/processed/social_all.csv', parse_dates=['date'])

In [4]:
# GS/lv 서울시만
gs_seoul = gs.loc[gs.pvn_nm =='서울특별시']
lv_seoul = lv.loc[lv.pvn_nm =='서울특별시']

In [5]:
# 컬럼 이름을 Series로 돌려주는 함수를 자체 개발함
sns_all.columns = changeColNames(sns_all,'.','_')
# 컬럼명의 마침표(.)가 전부 언더스코어(_)로 변경되었다.
# ols formula입력시 오류 발생 방지
# sns_all.columns
# 'p_wUVair_seoul.csv'파일 수정일:2019-07-11기준 자료로
# 수정된 자료로 저장되어있음.

In [6]:
cols_to_keep = ['date','bor_nm','gender','age_cd','category','qty']
gs_grouped = gs_seoul[cols_to_keep].groupby(by=['date','bor_nm','category']).sum().reset_index()
# gs_grouped.tail(2)

lv_grouped = lv_seoul[cols_to_keep].groupby(by=['date','bor_nm','category']).sum().reset_index()
# lv_grouped.tail(2)

In [8]:
# 서울특별시 단위로 df조정(구단위 데이터를 시단위로 합치기)
day_gs_grouped = gs_grouped.groupby(by=['date','category']).sum().reset_index()
# day_gs_grouped.tail(3)

day_lv_grouped = lv_grouped.groupby(by=['date','category']).sum().reset_index()
# day_lv_grouped.tail(3)

In [31]:
# '아이스크림'만 골라서 날씨('w')df와 소셜('sns_all')df와 결합해 새로운 'day_gs_grouped_w_item'생성
item = '아이스크림'
day_gs_grouped_w_item = mergeForAnalysis(day_gs_grouped, w, sns_all, item)
# day_gs_grouped_w_item = pd.merge(day_gs_grouped_w.loc[day_gs_grouped_w.category==item],w,on='date',how='left')
# day_gs_grouped_w_item = pd.merge(day_gs_grouped_w_item,sns_all,on='date',how='left')
# day_gs_grouped_w_item.tail(3)

In [32]:
# list_col = ['temp', 'cloud', 'wind', 'lgt_time','PM25',]
list_col = ['temp', 'wind', 'lgt_time','PM25',]
lowVIF(w,15,list_col)

,VIF_Factor,features
0,2.043588,temp
1,3.006264,PM25
2,3.958003,lgt_time
3,4.745850,wind


In [33]:
# ind_vars = ['temp','humid','wind','sun_time']
# ind_vars = ['temp','humid','wind','rain','snow','cloud','sun_time']
# ind_vars = ['temp','humid','wind','rain','snow','cloud','sun_time'
#                 ,'pm_total', 'health_total','br_total', 'hobby_total','date_total']

ind_vars = list_col
linReg(day_gs_grouped_w_item,item,ind_vars)
ridgeReg(day_gs_grouped_w_item,item,ind_vars)
lassoReg(day_gs_grouped_w_item,item,ind_vars)
rfReg(day_gs_grouped_w_item,item,ind_vars)
xgbReg(day_gs_grouped_w_item,item,ind_vars)

LinearRegression을 이용한 아이스크림의 회귀분석 결과 :
훈련세트점수 : 0.73
검증세트점수 : 0.73
RidgeRegression을 이용한 아이스크림의 회귀분석 결과 :
훈련세트점수 : 0.72
검증세트점수 : 0.72
LassoRegression을 이용한 아이스크림의 회귀분석 결과 :
훈련세트점수 : 0.73
검증세트점수 : 0.73
사용한 특성수 : 4
Random forest validation MAE =  3498.120668693009
훈련세트점수 : 0.98
검증세트점수 : 0.86
XGBoostRegression을 이용한 아이스크림의 회귀분석 결과 :
XGBoost validation MAE =  3304.918952733188
훈련세트점수 : 0.93
검증세트점수 : 0.87


In [34]:
Xy = day_gs_grouped_w_item.loc[day_gs_grouped_w_item['category']==item,ind_vars+['qty']]
customF = formulaGen('qty',ind_vars)
model = sm.OLS.from_formula(customF, data=Xy)

print(model.fit().summary())

                            OLS Regression Results                            
Dep. Variable:                    qty   R-squared:                       0.731
Model:                            OLS   Adj. R-squared:                  0.730
Method:                 Least Squares   F-statistic:                     740.6
Date:                Wed, 17 Jul 2019   Prob (F-statistic):          4.75e-309
Time:                        17:53:38   Log-Likelihood:                -11157.
No. Observations:                1096   AIC:                         2.232e+04
Df Residuals:                    1091   BIC:                         2.235e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   -514.5738    925.006     -0.556      0.5

In [35]:
gs_items = gs.category.unique()
lv_items = lv.category.unique()

In [36]:
# 한번에 처리 gs
print(list_col)
gs_scores = pd.DataFrame()
for item in gs_items:
#     print('\n',item,' 판매량 분석 - 모든 독립변수 포함(날씨+sns(합계))')
#     df_working = mergeForAnalysis(day_gs_grouped, w, sns_all, item)
    temp_df 
    print('\n',item,' 판매량 분석 - VIF작은거 5개만 ')
    df_working = mergeForAnalysis(day_gs_grouped, w, item=item)
    tr_score, test_score = linReg(df_working,item,ind_vars,score=True)
    gs_scores['물품'] = item
    ridgeReg(df_working,item,ind_vars,score=True)
    lassoReg(df_working,item,ind_vars,score=True)
    rfReg(df_working,item,ind_vars,score=True)
    xgbReg(df_working,item,ind_vars,score=True)
    print()

['temp', 'wind', 'lgt_time', 'PM25']

 라면  판매량 분석 - VIF작은거 5개만 
LinearRegression을 이용한 라면의 회귀분석 결과 :
훈련세트점수 : 0.16
검증세트점수 : 0.12
RidgeRegression을 이용한 라면의 회귀분석 결과 :
훈련세트점수 : 0.16
검증세트점수 : 0.12
LassoRegression을 이용한 라면의 회귀분석 결과 :
훈련세트점수 : 0.16
검증세트점수 : 0.12
사용한 특성수 : 4
Random forest validation MAE =  1911.0759878419453
훈련세트점수 : 0.85
검증세트점수 : 0.10
XGBoostRegression을 이용한 라면의 회귀분석 결과 :
XGBoost validation MAE =  1865.754577080167
훈련세트점수 : 0.47
검증세트점수 : 0.12


 과자  판매량 분석 - VIF작은거 5개만 
LinearRegression을 이용한 과자의 회귀분석 결과 :
훈련세트점수 : 0.11
검증세트점수 : 0.06
RidgeRegression을 이용한 과자의 회귀분석 결과 :
훈련세트점수 : 0.11
검증세트점수 : 0.06
LassoRegression을 이용한 과자의 회귀분석 결과 :
훈련세트점수 : 0.11
검증세트점수 : 0.06
사용한 특성수 : 4
Random forest validation MAE =  5342.733130699088
훈련세트점수 : 0.83
검증세트점수 : -0.06
XGBoostRegression을 이용한 과자의 회귀분석 결과 :
XGBoost validation MAE =  5077.232718702508
훈련세트점수 : 0.42
검증세트점수 : 0.07


 마스크  판매량 분석 - VIF작은거 5개만 
LinearRegression을 이용한 마스크의 회귀분석 결과 :
훈련세트점수 : 0.25
검증세트점수 : 0.15
RidgeRegression을 이용한 마스크의 회귀분석 결과 

In [37]:
# 한번에 처리 lv
print(list_col)

for item in lv_items:
    print('\n',item,' 판매량 분석 - 모든 독립변수 포함')
    df_working = mergeForAnalysis(day_lv_grouped, w, item=item)
    linReg(df_working,item,ind_vars)
    ridgeReg(df_working,item,ind_vars)
    lassoReg(df_working,item,ind_vars)
    rfReg(df_working,item,ind_vars)
    xgbReg(df_working,item,ind_vars)

['temp', 'wind', 'lgt_time', 'PM25']

 립컬러  판매량 분석 - 모든 독립변수 포함
LinearRegression을 이용한 립컬러의 회귀분석 결과 :
훈련세트점수 : 0.02
검증세트점수 : 0.01
RidgeRegression을 이용한 립컬러의 회귀분석 결과 :
훈련세트점수 : 0.02
검증세트점수 : 0.01
LassoRegression을 이용한 립컬러의 회귀분석 결과 :
훈련세트점수 : 0.02
검증세트점수 : 0.01
사용한 특성수 : 4
Random forest validation MAE =  6716.045896656536
훈련세트점수 : 0.79
검증세트점수 : -0.09
XGBoostRegression을 이용한 립컬러의 회귀분석 결과 :
XGBoost validation MAE =  6181.15594278353
훈련세트점수 : 0.52
검증세트점수 : 0.01

 립케어  판매량 분석 - 모든 독립변수 포함
LinearRegression을 이용한 립케어의 회귀분석 결과 :
훈련세트점수 : 0.23
검증세트점수 : 0.23
RidgeRegression을 이용한 립케어의 회귀분석 결과 :
훈련세트점수 : 0.23
검증세트점수 : 0.23
LassoRegression을 이용한 립케어의 회귀분석 결과 :
훈련세트점수 : 0.23
검증세트점수 : 0.23
사용한 특성수 : 4
Random forest validation MAE =  2445.3045592705166
훈련세트점수 : 0.85
검증세트점수 : 0.11
XGBoostRegression을 이용한 립케어의 회귀분석 결과 :
XGBoost validation MAE =  2154.6541794426175
훈련세트점수 : 0.57
검증세트점수 : 0.25

 마스크팩  판매량 분석 - 모든 독립변수 포함
LinearRegression을 이용한 마스크팩의 회귀분석 결과 :
훈련세트점수 : 0.01
검증세트점수 : -0.00
RidgeRegression을 이용한 마스크팩의